In [18]:
import sys

In [19]:
'''
入力形式：
1行目：座席の行数，列数
2行目：学生の総数
3行目以降：
各行ごとに
    名前と選好度(ここでは1～5とする)
以下，.txtでの読み込みを想定
'''

info = ('name','Blackboard','Window','Airconditioner','Edge') #選好に対応する命名

#rawのデータをsummarizedにまとめる
def summarize(raw):
    with open(raw, encoding = 'utf-8') as f:
        row, column = map(int, f.readline().split()) #1行目：座席の行数，列数
        total = int(f.readline()) #2行目：学生の総数

        #3行目以降
        students = [] #個人データ(名前と選好項目）
        #1行ずつ全員分の個人データを読み込む
        for i in range(total):
            data = f.readline().split()
            data[1:] = map(int, data[1:]) #先頭のname以外の要素をint型に変換
            students.append(dict(zip(info, data))) #infoをkeyにして個人データを1人分ずつ辞書にまとめる

    summarized = (row, column, total, students)
    return summarized

#raw_dataを受け取り，データをsummaized_dataにまとめる
#raw_data = sys.argv[1] #サンプルデータはコマンドラインで指定
raw_data = 'input_sample2.txt' #txtファイルを入力
summarized_data = summarize(raw_data)

In [20]:
########################################################################################################################################################################
#ここから前準備
########################################################################################################################################################################

# 学生番号の集合を定義：学生N人
N = summarized_data[2] #total
I = [i for i in range(N)]

# 席位置の集合を定義：縦K行、横L列
K = summarized_data[0] #row
L = summarized_data[1] #column
M = K*L #席数
J = [j for j in range(M)]
# 前方左側を0，そこから右に1,2,...と番号を振る．
# j\in Jは前からj//L番目，左からj%p番目の席を表す．

#それぞれの席における距離や、属性を定義
#distance_from_blackboard = {}
Window = {}
Airconditioner = {}
Edge = {}
u_func_BB = {}

for i in range(K):
    for j in range(L):
        u_func_BB[i*L + j] = 1/(i+1)
        #distance_from_blackboard[i*L + j] = i + 1

#その席が属性に対応しているか0or1
for i in range(K):
    for j in range(L):
        if j==0:
            Window[i*L + j] = 1
        else:
            Window[i*L + j] = 0

for i in range(K):
    for j in range(L):
        if j== L-3:
            Airconditioner[i*L + j] = 1
        else:
            Airconditioner[i*L + j] = 0

for i in range(K):
    for j in range(L):
        if j==0 or j==L/2-1 or j == L/2 or j == L-1:
            Edge[i*L + j] = 1
        else:
            Edge[i*L + j] = 0

In [22]:
########################################################################################################################################################################
#ここから最適化問題の係数を作るところ
########################################################################################################################################################################

#効用関数とそれぞれの選好を定義
u_Blackboard = [summarized_data[3][i][info[1]] for i in range(N)]
u_Window = [summarized_data[3][i][info[2]] for i in range(N)]
u_Airconditioner = [summarized_data[3][i][info[3]] for i in range(N)]
u_Edge = [summarized_data[3][i][info[4]] for i in range(N)]


#目的関数の係数となるCを定義
c_Blackboard = {}
c_Window = {}
c_Airconditioner = {}
c_Edge = {}

for i in range(N):
    for j in range(M):
        c_Blackboard[i,j] = u_Blackboard[i] * u_func_BB[j]

for i in range(N):
    for j in range(M):
        c_Window[i,j] = u_Window[i] * Window[j]

for i in range(N):
    for j in range(M):
        c_Airconditioner[i,j] = u_Airconditioner[i] * Airconditioner[j]

for i in range(N):
    for j in range(M):
        c_Edge[i,j] = u_Edge[i] * Edge[j]


# 各個人の位置だけで定まる選好 c_1,c_2,...
#ここを完成させたい
c = {}
for i in range(N):
    for j in range(M):
        c[i,j] = c_Blackboard[i,j] + c_Window[i,j] - c_Airconditioner[i,j] + c_Edge[i,j]

In [41]:
import numpy as np

In [42]:
c_np_array = np.zeros((N, M)) #Nが学生の数、Mが席の数

In [43]:
for i in range(N):
    for j in range(M):
        c_np_array[i][j] = c[i,j]

In [45]:
df = pd.DataFrame(c_np_array)

In [48]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,5.0,1.0,1.0,1.0,3.0,3.0,1.0,-2.0,1.0,3.0,...,4.2,0.2,0.2,0.2,2.2,2.2,0.2,-2.8,0.2,2.2
1,9.0,3.0,3.0,3.0,6.0,6.0,3.0,2.0,3.0,6.0,...,6.6,0.6,0.6,0.6,3.6,3.6,0.6,-0.4,0.6,3.6
2,8.0,3.0,3.0,3.0,6.0,6.0,3.0,1.0,3.0,6.0,...,5.6,0.6,0.6,0.6,3.6,3.6,0.6,-1.4,0.6,3.6
3,4.0,1.0,1.0,1.0,3.0,3.0,1.0,0.0,1.0,3.0,...,3.2,0.2,0.2,0.2,2.2,2.2,0.2,-0.8,0.2,2.2
4,6.0,1.0,1.0,1.0,3.0,3.0,1.0,-2.0,1.0,3.0,...,5.2,0.2,0.2,0.2,2.2,2.2,0.2,-2.8,0.2,2.2


In [49]:
df.to_csv("c_df_0605_1.csv")

In [52]:
print(K)

5


In [ ]:
'''
5 10
20

#各人の選好
[2,3,4,1,-1,3]

#出てきた解をそのまま渡す
[[],[],[]]
'''